In [1]:
pip install nltk

     ---------------------------------------- 1.5/1.5 MB 10.6 MB/s eta 0:00:00
     ---------------------------------------- 96.6/96.6 kB ? eta 0:00:00
     ------------------------------------- 267.7/267.7 kB 16.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts

In [2]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, svm

In [3]:
data = pd.read_csv('mu_data.csv')
data.head()

,id,Polls ID,Assessment reports Student ID,Poll Responses Response,Assessment reports Hashtag,Assessment reports Score
0,1,12522.0,41.0,The strengths of Plato's approach is his const...,#objectivemorality,2.0
1,2,12522.0,42.0,Having a difficulty with his Divided Line. Th...,#deductivearg,2.0
2,3,12522.0,44.0,"How can one know if he's in the cave or not, i...",#objectivemorality,3.0
3,4,12522.0,237.0,S: the hierarchy of the chart we filled out at...,#objectivemorality,3.0
4,5,12522.0,239.0,It is difficult to grasp how to apply Plato's ...,#objectivemorality,3.0


In [4]:
data = data.rename(columns={
    'Polls ID': 'poll_id',
    'Assessment reports Student ID': 'student_id',
    'Poll Responses Response':'response',
    'Assessment reports Hashtag':'HC',
    'Assessment reports Score':'score'})
data.head()

,id,poll_id,student_id,response,HC,score
0,1,12522.0,41.0,The strengths of Plato's approach is his const...,#objectivemorality,2.0
1,2,12522.0,42.0,Having a difficulty with his Divided Line. Th...,#deductivearg,2.0
2,3,12522.0,44.0,"How can one know if he's in the cave or not, i...",#objectivemorality,3.0
3,4,12522.0,237.0,S: the hierarchy of the chart we filled out at...,#objectivemorality,3.0
4,5,12522.0,239.0,It is difficult to grasp how to apply Plato's ...,#objectivemorality,3.0


In [19]:
deduction = data[data['HC'] == '#deduction']
deduction.shape
deduction.head()

,id,poll_id,student_id,response,HC,score
263,264,19704.0,283.0,Freshwater economists would state that the bub...,#deduction,4.0
626,627,20862.0,625.0,The most important moment of class for me was ...,#deduction,3.0
627,628,20862.0,634.0,"Free choice can be manipulated with language, ...",#deduction,3.0
628,629,20862.0,653.0,The activities regarding quantifiers placed a ...,#deduction,3.0
629,630,20862.0,673.0,Applying quantifiers that eliminate the ambigu...,#deduction,3.0


In [6]:
"""
Rubric for #deduction
1. Accurately applies and analyzes deductive reasoning with clear and detailed explanation
2. accurately identifies premises and conclusions and justifies the identification
3. provides a detailed, well-justified evaluation of the relationships among propositions with easy to follow steps
4. accurately distinguishes between validity and soundness and explains the meaning of each in the given context
5. provides a detailed, well justified evaluation of the validity or soundness
6. accurately and effectively applies deductive techniques to a complex, sophisticated derivation.

Assuming that a person gets a 4 if they applied at least half of these, 3 if they applied 2, 2 if they applied 1 
"""

'\nRubric for #deduction\n1. Accurately applies and analyzes deductive reasoning with clear and detailed explanation\n2. accurately identifies premises and conclusions and justifies the identification\n3. provides a detailed, well-justified evaluation of the relationships among propositions with easy to follow steps\n4. accurately distinguishes between validity and soundness and explains the meaning of each in the given context\n5. provides a detailed, well justified evaluation of the validity or soundness\n6. accurately and effectively applies deductive techniques to a complex, sophisticated derivation.\n\nAssuming that a person gets a 4 if they applied at least half of these, 3 if they applied 2, 2 if they applied 1 \n'

In [26]:
deduction = deduction.drop(['id', 'student_id', 'HC'], axis = 1)


In [8]:
rubric = np.zeros((2977, 6))
i = 0
for score in deduction['score']:
    if score == 4.0:
        c = np.random.randint(4)
        rubric[i] = np.random.shuffle([0]*c + [1]*(c-1))
    elif score == 3.0:
        c = np.random.randint(5)
        rubric[i] = np.random.shuffle([0]*c + [1]*(c-1))

#np.random.randint(2, size = (10, 6))


In [33]:
np.random.randint(4)

2